In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import cadquery as cq
from cq_jupyter import Assembly, Part, Edges, Faces, display, exportSTL

# CadQuery Example

In [4]:
b = cq.Workplane('XY')
box1 = b.box(1,2,3).edges(">X or <X").chamfer(0.2)
box2 = b.transformed(offset=cq.Vector(0, 1.5, 0.7)).box(3,2,0.6)\
        .edges(">Z").fillet(0.3)
box3 = b.transformed(offset=cq.Vector(0, 1.0, -0.7)).box(2,3,0.6)\
        .edges("<Z").fillet(0.3)
box1.cut(box2)
box1.cut(box3)

View id: 6b9f09dc-0766-4675-8b6f-4766924c4bac


# NEW APPROACH

## ImageButton Widget

## Rendering

In [7]:
from __future__ import print_function, absolute_import

import sys
import enum
import uuid
import operator
from functools import reduce

from pythreejs import *
from IPython.display import display
from ipywidgets import HTML, HBox, VBox
import numpy as np

from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib_Add
from OCC.Core.Visualization import Tesselator

from OCC.Extend.TopologyUtils import TopologyExplorer, is_edge, is_wire, discretize_edge, discretize_wire

/opt/miniconda/envs/pycq/lib/python3.6/site-packages/pythreejs/traits.py:55: DeprecationWarning: Traits should be given as instances, not types (for example, `Int()`, not `Int`) Passing types is deprecated in traitlets 4.1.
  super(Vector3, self).__init__(*(trait, trait, trait), default_value=default_value, **kwargs)
/opt/miniconda/envs/pycq/lib/python3.6/site-packages/pythreejs/pythreejs.py:56: DeprecationWarning: Traits should be given as instances, not types (for example, `Int()`, not `Int`). Passing types is deprecated in traitlets 4.1.
  z = List(CFloat, [0] * 100)


In [8]:
HAVE_SMESH = False

# default values

def format_color(r, g, b):
    return '#%02x%02x%02x' % (r, g, b)

default_shape_color = format_color(166, 166, 166)
default_mesh_color = 'white'
default_edge_color = format_color(0, 0, 0)
default_selection_material = MeshPhongMaterial(color='orange',
                                               polygonOffset=True,
                                               polygonOffsetFactor=1,
                                               polygonOffsetUnits=1,
                                               shininess=0.9,
                                               side='DoubleSide')

class NORMAL(enum.Enum):
    SERVER_SIDE = 1
    CLIENT_SIDE = 2
    
_compute_normals_mode = NORMAL.SERVER_SIDE

In [9]:
def AddShapeToScene(shp,  # the TopoDS_Shape to be displayed
                    shape_color=default_shape_color,  # the default
                    render_edges=False,
                    edge_color=default_edge_color,
                    compute_uv_coords=False,
                    quality=1.0,
                    transparency=True,
                    opacity=0.6):
    # first, compute the tesselation
    tess = Tesselator(shp)
    tess.Compute(uv_coords=compute_uv_coords,
                 compute_edges=render_edges,
                 mesh_quality=quality,
                 parallel=True)
    
    # get vertices and normals
    vertices_position = tess.GetVerticesPositionAsTuple()

    number_of_triangles = tess.ObjGetTriangleCount()
    number_of_vertices = len(vertices_position)

    # number of vertices should be a multiple of 3
    if number_of_vertices % 3 != 0:
        raise AssertionError("Wrong number of vertices")
    if number_of_triangles * 9 != number_of_vertices:
        raise AssertionError("Wrong number of triangles")

    # then we build the vertex and faces collections as numpy ndarrays
    np_vertices = np.array(vertices_position, dtype='float32').reshape(int(number_of_vertices / 3), 3)
    # Note: np_faces is just [0, 1, 2, 3, 4, 5, ...], thus arange is used
    np_faces = np.arange(np_vertices.shape[0], dtype='uint32')

    # set geometry properties
    buffer_geometry_properties = {'position': BufferAttribute(np_vertices),
                                  'index'   : BufferAttribute(np_faces)}
    if _compute_normals_mode == NORMAL.SERVER_SIDE:
        # get the normal list, converts to a numpy ndarray. This should not raise
        # any issue, since normals have been computed by the server, and are available
        # as a list of floats
        np_normals = np.array(tess.GetNormalsAsTuple(), dtype='float32').reshape(-1, 3)
        # quick check
        if np_normals.shape != np_vertices.shape:
            raise AssertionError("Wrong number of normals/shapes")
        buffer_geometry_properties['normal'] = BufferAttribute(np_normals)

    # build a BufferGeometry instance
    shape_geometry = BufferGeometry(attributes=buffer_geometry_properties)

    # if the client has to render normals, add the related js instructions
    if _compute_normals_mode == NORMAL.CLIENT_SIDE:
        shape_geometry.exec_three_obj_method('computeVertexNormals')

    # then a default material
    shp_material = MeshPhongMaterial(color=shape_color,
                                     polygonOffset=True,
                                     polygonOffsetFactor=1,
                                     polygonOffsetUnits=1,
                                     shininess=0.9,
                                     transparent=transparency,
                                     opacity=opacity,
                                     side='DoubleSide')

    # create a mesh unique id
    mesh_id = uuid.uuid4().hex

    # finally create the mash
    shape_mesh = Mesh(geometry=shape_geometry,
                      material=shp_material,
                      name=mesh_id)

    # edge rendering, if set to True
    def explode(edge_list):
        return [[edge_list[i], edge_list[i+1]] for i in range(len(edge_list)-1)]

    def flatten(nested_list):
        return [y for x in nested_list for y in x]
    
    
    edge_lines = None
    if render_edges:
        edges = list(map(lambda i_edge: [tess.GetEdgeVertex(i_edge, i_vert) 
                                         for i_vert in range(tess.ObjEdgeGetVertexCount(i_edge))], 
                         range(tess.ObjGetEdgeCount())))
        edges = flatten(list(map(explode, edges)))
        np_edge_vertices = np.array(edges, dtype=np.float32).reshape(-1, 3)
        np_edge_indices = np.arange(np_edge_vertices.shape[0], dtype=np.uint32)
        edge_geometry = BufferGeometry(attributes={
            'position': BufferAttribute(np_edge_vertices),
            'index'   : BufferAttribute(np_edge_indices)
        })
        edge_material = LineBasicMaterial(color=edge_color, linewidth=1)
        edge_lines = LineSegments(geometry=edge_geometry, material=edge_material)

    # Add geometries to pickable or non pickable objects
    return (shape_mesh, edge_lines)


In [85]:
b.center.toTuple()

(0.0, 0.7499999999999999, 0.0)

In [89]:
import math 
from cadquery import Compound

def bbox(shapes):
    compounds = [Compound.makeCompound(shape.objects) for shape in shapes]
    return Compound.makeCompound(compounds).BoundingBox()

view_width = 600
view_height = 400

mesh1, edges1 = AddShapeToScene(box1.toOCC(), render_edges=True, shape_color="#ff0000")
mesh2, edges2 = AddShapeToScene(box2.toOCC(), render_edges=True, shape_color="#00ff00")
mesh3, edges3 = AddShapeToScene(box3.toOCC(), render_edges=True, shape_color="#0000ff")

bb = bbox([box1, box2, box3])

axes = AxesHelper(3)
grid = GridHelper(6,12)
grid.rotation = (math.pi / 2.0, 0, 0, "XYZ")

key_light = PointLight(position=[-100, 100, 100])
ambient_light = AmbientLight(intensity=0.4)

scene = Scene(children=[axes, grid, 
                        mesh1, edges1, 
                        mesh2, edges2, 
                        mesh3, edges3, 
                        key_light, ambient_light, camera])

all_shapes = scene.children[2:-3]

camera_target = bb.center.toTuple()
camera_position = [0, 0, bb.DiagonalLength]

camera = CombinedCamera(position=camera_position, width=view_width, height=view_height)


orbit = OrbitControls(controlling=camera, target=camera_target, minPolarAngle=-math.inf, maxPolarAngle=math.inf)

renderer = Renderer(scene=scene, camera=camera, controls=[orbit],
                    width=view_width, height=view_height)

In [92]:
from ipywidgets import ToggleButton, Button, Checkbox, Layout

orientations = ["fit", "isometric", "right", "front", "left", "rear", "top", "bottom"]
directions = {
    "left" :     ( 1,  0,  0),
    "right":     (-1,  0,  0),
    "front":     ( 0, -1,  0),
    "rear" :     ( 0,  1,  0),
    "top":       ( 0,  0,  1),
    "bottom":    ( 0,  0, -1),
    "isometric": ( 1,  1, -1)
}

def load_image(d):
    return open("/Users/bernhard/Development/cadquery/cadquery-jupyter-extension/icons/%s.png" % d, 'rb').read()

def im_button(d, handler): 
    button = ImageButton(
        width=36, 
        height=28, 
        value=load_image(d),
        tooltip=d,
        typ=d
    )
    button.on_click(handler)
    return button

def scale(vec, r):
    n = np.linalg.norm(np.array(vec))
    return (vec / n * r).tolist()


def handle(b):    
    if b.typ == "fit":
        renderer.camera.position = scale(renderer.camera.position, bb.DiagonalLength)
    else:
        renderer.camera.position = scale(directions[b.typ], bb.DiagonalLength)

    controller = renderer.controls[0]
    controller.exec_three_obj_method('update')

def toggleAxisGrid(i, b):
    def f(change):
        scene.children[i].visible = change["new"]
    return f

def toggleOrtho(camera):
    def f(change):
        if change["new"]:
            camera.mode = 'orthographic'
        else:
            camera.mode = 'perspective'
    return f

def sceneChecks():
    checks = []
    for i, name in ((0, "Axis"), (1, "Grid"), (2, "Ortho")):
        checks.append(Checkbox(value=True, description=name, indent=False, 
                               layout=Layout(width="65px")))
        if name == "Ortho":
            checks[-1].observe(toggleOrtho(camera), "value")
        else:
            checks[-1].observe(toggleAxisGrid(i, buttons[i]), "value")
        scene.children[i].visible = True
    return checks


buttons = [im_button(d, handle) for d in orientations]

buttons += sceneChecks()

In [93]:
VBox([HBox(buttons), renderer])

In [ ]:
orbit.

In [ ]:
scene.children[2].visible = True

In [ ]:
renderer.camera.position = scale((0, 0, 1) ,b.DiagonalLength)
renderer.controls[0].exec_three_obj_method('update')
camera.quaternion

In [ ]:
camera.quaternion = (-2.164890140588733e-17,
 -0.7071067811865475,
 -2.164890140588733e-17,
 0.7071067811865476)

#(0.279848, 0.115917, 0.364705, 0.880476)
renderer.controls[0].exec_three_obj_method('update')

In [ ]:
camera.quaternion 

In [46]:
orbit.maxAzimuthAngle

inf

In [ ]:
from pythreejs import Quaternion
from math import atan, sqrt

q1 = Quaternion(1, 0, 0, atan(1 / sqrt(2)))
q2 = Quaternion(0, 0, 1, pi / 4)

from OCC.gp import gp_Quaternion, gp_Vec
q1 = gp_Quaternion(gp_Vec(1, 0, 0), atan(1 / sqrt(2)))
q2 = gp_Quaternion(gp_Vec(0, 0, 1), pi / 4)
q2 * q1

# ====== SCRATCH ======

In [ ]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

In [ ]:
from ipywidgets import Image, Button, HBox, VBox
from IPython.display import display, Javascript

def scale(vec, r):
    n = np.linalg.norm(np.array(vec))
    return (vec / n * r).tolist()

def im(d): 
    image = Image(width=40, 
                  height=32, 
                  format='png',
                  value=open("/Users/bernhard/Development/cadquery/cadquery-jupyter-extension/icons/%s.png" % d, 'rb').read())
    image.add_class('im_control')
    image.add_class(d)
    return image

buttons = [im(d) for d in ["fit", "isometric", "right", "front", "left", "rear", "top", "bottom"]]

In [ ]:
directions = {
    "left" :     ( 1, 0, 0),
    "right":     (-1, 0, 0),
    "front":     ( 0, 1, 0),
    "rear" :     ( 0,-1, 0),
    "top":       ( 0, 0, 1),
    "bottom":    ( 0, 0,-1),
    "isometric": ( 1, 1, -1)
}

def move_camera(d):
    if d == "fit":
        my_renderer._camera.position = scale(my_renderer._camera.position, b.DiagonalLength)
    else:
        my_renderer._camera.position = scale(directions[d] ,b.DiagonalLength)

    controller = my_renderer._renderer.controls[0]
    controller.exec_three_obj_method('update')

def on_image_click(icon):
    move_camera(icon)
    return 

In [ ]:
class JupyterRenderer(object):
    def __init__(self, size=(480, 480), compute_normals_mode=NORMAL.SERVER_SIDE, parallel=False):
        """ Creates a jupyter renderer.
        size: a tuple (width, height). Must be a square, or shapes will look like deformed
        compute_normals_mode: optional, set to SERVER_SIDE by default. This flag lets you choose the
        way normals are computed. If SERVER_SIDE is selected (default value), then normals
        will be computed by the Tesselator, packed as a python tuple, and send as a json structure
        to the client. If, on the other hand, CLIENT_SIDE is chose, then the computer only compute vertex
        indices, and let the normals be computed by the client (the web js machine embedded in the webrowser).

        * SERVER_SIDE: higher server load, loading time increased, lower client load. Poor performance client will
          choose this option (mobile terminals for instance)
        * CLIENT_SIDE: lower server load, loading time decreased, higher client load. Higher performance clients will
                            choose this option (laptops, desktop machines).
        * parallel: optional, False by default. If set to True, meshing runs in parallelized mode.
        """
        self._background = 'white'
        self._background_opacity = 1
        self._size = size
        self._compute_normals_mode = compute_normals_mode
        self._parallel = parallel

        self.html = HTML("Selected shape : None")
        # the default camera object
        self._camera_target = [0., 0., 0.]  # the point to look at
        self._camera_position = [0, 0., 100.]  # the camera initial position
        self._camera = None

        # a dictionnary of all the shapes belonging to the renderer
        # each element is a key 'mesh_id:shape'
        self._shapes = {}

        # we save the renderer so that is can be accessed
        self._renderer = None

        # the group of 3d and 2d objects to render
        self._displayed_pickable_objects = Group()

        # the group of other objects (grid, trihedron etc.) that can't e selected
        self._displayed_non_pickable_objects = Group()

        # event manager/selection manager
        self._picker = Picker(controlling=self._displayed_pickable_objects, event='mousedown')

        self._current_shape_selection = None
        self._current_mesh_selection = None
        self._current_selection_material = None  # the color of the object currently being rendered
        self._select_callbacks = []  # a list of all functions called after an object is selected


        def click(value):
            """ called whenever a shape  or edge is clicked
            """
            obj = value.owner.object
            if self._current_mesh_selection is not None:
                self._current_mesh_selection.material = self._current_selection_material
            if obj is not None:
                id_clicked = obj.name  # the mesh id clicked
                self._current_mesh_selection = obj
                self._current_selection_material = obj.material
                obj.material = default_selection_material
                # get the shape from this mesh id
                selected_shape = self._shapes[id_clicked]
                self.html.value = "<b>Shape id:</b> %s<br><b>Topology hierearchy</b>" % (selected_shape)
                self._current_shape_selection = selected_shape
            else:
                self.html.value = "<b>Shape id:</b> None"
            # then execute calbacks
            for callback in self._select_callbacks:
                callback(self._current_shape_selection)

        self._picker.observe(click)

        # key press and related events
        #def key_pressed(widget):
        #    print("key pressed")
        #self._picker2 = Picker(event='keypress')
        #self._picker.observe(key_pressed)


    def register_select_callback(self, callback):
        """ Adds a callback that will be called each time a shape is selected
        """
        if not callable(callback):
            raise AssertionError("You must provide a callable to register the callback")
        else:
            self._select_callbacks.append(callback)


    def unregister_callback(self, callback):
        """ Remove a callback from the callback list
        """
        if not callback in self._select_callbacks:
            raise AssertionError("This callback is not registered")
        else:
            self._select_callbacks.remove(callback)


    def _update_camera(self):
        all_shapes = list(self._shapes.values())
        if all_shapes:
            bb = reduce(operator.add, map(bounding_box, all_shapes))
            print([bb.x_center, bb.y_center, bb.z_center])
            print([bb.x_size, bb.y_size, bb.z_size])
            self._camera_target = [bb.x_center, bb.y_center, bb.z_center]
            self._camera_position = [0, bb.y_center - 2 * bb.y_size, bb.z_center + 2 * bb.z_center]
        self._camera = OrthographicCamera(left=-bb.y_size, 
                                          right=bb.y_size, 
                                          top=bb.z_size, 
                                          bottom=-bb.z_size,
                                          children=[DirectionalLight(color='#ffffff', position=[50, 50, 50], intensity=0.9)])
#         self._camera = PerspectiveCamera(position=self._camera_position,
#                                          lookAt=self._camera_target,
#                                          up=[0, 0, 1],
#                                          aspect=self._size[0] / self._size[1],
#                                          fov=50,
#                                          children=[DirectionalLight(color='#ffffff', position=[50, 50, 50], intensity=0.9)])


    def GetSelectedShape(self):
        """ Returns the selected shape
        """
        return self._current_shape_selection


    def DisplayGrid(self, sizex, sizey, nx, ny):
        """ Displays a grid in the renderer.
        sizex: float, grid size along x axis
        sizey: float, grid size along y axis
        nx: integer, number of segments along the x axis
        ny: integer, number of segments along the y axis
        """
        surf_geo = SurfaceGeometry(z=[0] * (nx + 1) * (ny + 1),
                                   width=sizex,
                                   height=sizey,
                                   width_segments=nx,
                                   height_segments=ny)
        surf_grid = SurfaceGrid(geometry=surf_geo,
                                material=LineBasicMaterial(color='#000000',
                                                           opacity=0.2,
                                                           transparent=True))
        self._displayed_non_pickable_objects.add(surf_grid)
        return surf_geo


    def DisplayMesh(self,
                    mesh,
                    color=default_mesh_color):
        """ Display a MEFISTO2 triangle mesh
        """
        if not HAVE_SMESH:
            print("SMESH not installed, DisplayMesh method unavailable.")
            return
        if not isinstance(mesh, SMESH_Mesh):
            raise AssertionError("You mush provide an SMESH_Mesh instance")
        mesh_ds = mesh.GetMeshDS()  # the mesh data source
        face_iter = mesh_ds.facesIterator()
        # vertices positions are stored to a liste
        vertices_position = []
        for _ in range(mesh_ds.NbFaces()-1):
            face = face_iter.next()
            #print('Face %i, type %i' % (i, face.GetType()))
            #print(dir(face))
            # if face.GetType == 3 : triangle mesh, then 3 nodes
            for j in range(3):
                node = face.GetNode(j)
                #print('Coordinates of node %i:(%f,%f,%f)'%(i, node.X(), node.Y(), node.Z()))
                vertices_position.append(node.X())
                vertices_position.append(node.Y())
                vertices_position.append(node.Z())
        number_of_vertices = len(vertices_position)
        # then we build the vertex and faces collections as numpy ndarrays
        np_vertices = np.array(vertices_position, dtype='float32').reshape(int(number_of_vertices / 3), 3)
        # Note: np_faces is just [0, 1, 2, 3, 4, 5, ...], thus arange is used
        np_faces = np.arange(np_vertices.shape[0], dtype='uint32')
        # set geometry properties
        buffer_geometry_properties = {'position': BufferAttribute(np_vertices),
                                      'index'   : BufferAttribute(np_faces)}
        # build a BufferGeometry instance
        mesh_geometry = BufferGeometry(attributes=buffer_geometry_properties)

        mesh_geometry.exec_three_obj_method('computeVertexNormals')

        # then a default material
        mesh_material = MeshPhongMaterial(color=color,
                                          polygonOffset=True,
                                          polygonOffsetFactor=1,
                                          polygonOffsetUnits=1,
                                          shininess=0.5,
                                          wireframe=False,
                                          side='DoubleSide')
        edges_material = MeshPhongMaterial(color='black',
                                           polygonOffset=True,
                                           polygonOffsetFactor=1,
                                           polygonOffsetUnits=1,
                                           shininess=0.5,
                                           wireframe=True)
        # create a mesh unique id
        mesh_id = uuid.uuid4().hex

        # finally create the mash
        shape_mesh = Mesh(geometry=mesh_geometry,
                          material=mesh_material,
                          name=mesh_id)
        edges_mesh = Mesh(geometry=mesh_geometry,
                          material=edges_material,
                          name=mesh_id)


        # a special display for the mesh
        camera_target = [0., 0., 0.]  # the point to look at
        camera_position = [0, 0., 100.]  # the camera initial position
        self._camera = OrthographicCamera(left=-bb.y_size, 
                                          right=bb.y_size, 
                                          top=bb.z_size, 
                                          bottom=-bb.z_size,
                                          children=[DirectionalLight(color='#ffffff', position=[50, 50, 50], intensity=0.9)])
#         camera = PerspectiveCamera(position=camera_position,
#                                    lookAt=camera_target,
#                                    up=[0, 0, 1],
#                                    fov=50,
#                                    children=[DirectionalLight(color='#ffffff',
#                                                               position=[50, 50, 50],
#                                                               intensity=0.9)])
        scene_shp = Scene(children=[shape_mesh, edges_mesh, camera, AmbientLight(color='#101010')])

        renderer = Renderer(camera=camera,
                            background=self._background,
                            background_opacity=self._background_opacity,
                            scene=scene_shp,
                            controls=[OrbitControls(controlling=camera, target=camera_target)],
                            width=self._size[0],
                            height=self._size[1],
                            antialias=True)

        display(renderer)


    def DisplayShape(self,
                     shp,  # the TopoDS_Shape to be displayed
                     shape_color=default_shape_color,  # the default
                     render_edges=False,
                     edge_color=default_edge_color,
                     compute_uv_coords=False,
                     quality=1.0,
                     transparency=False,
                     opacity=1.,
                     topo_level='default',
                     update=False):
        """ Displays a topods_shape in the renderer instance.
        shp: the TopoDS_Shape to render
        shape_color: the shape color, in html corm, eg '#abe000'
        render_edges: optional, False by default. If True, compute and dislay all
                      edges as a linear interpolation of segments.
        edge_color: optional, black by default. The color used for edge rendering,
                    in html form eg '#ff00ee'
        compute_uv_coords: optional, false by default. If True, compute texture
                           coordinates (required if the shape has to be textured)
        quality: optional, 1.0 by default. If set to something lower than 1.0,
                      mesh will be more precise. If set to something higher than 1.0,
                      mesh will be less precise, i.e. lower numer of triangles.
        transparency: optional, False by default (opaque).
        opacity: optional, float, by default to 1 (opaque). if transparency is set to True,
                 0. is fully opaque, 1. is fully transparent.
        topo_level: "default" by default. The value should be either "compound", "shape", "vertex".
        update: optional, False by default. If True, render all the shapes.
        """
        if is_wire(shp) or is_edge(shp):
            self.AddCurveToScene(shp, shape_color)
        if topo_level != "default":
            t = TopologyExplorer(shp)
            map_type_and_methods = {"Solid": t.solids, "Face": t.faces, "Shell": t.shells,
                                    "Compound": t.compounds, "Compsolid": t.comp_solids}
            for subshape in map_type_and_methods[topo_level]():
                self.AddShapeToScene(subshape, shape_color, render_edges, edge_color, compute_uv_coords, quality,
                                     transparency, opacity)
        else:
            self.AddShapeToScene(shp, shape_color, render_edges, edge_color, compute_uv_coords, quality,
                                 transparency, opacity)
        if update:
            self.Display()


    def AddCurveToScene(self, shp, color):
        """ shp is either a TopoDS_Wire or a TopodS_Edge.
        """
        if is_edge(shp):
            pnts = discretize_edge(shp)
        elif is_wire(shp):
            pnts = discretize_wire(shp)
        np_edge_vertices = np.array(pnts, dtype=np.float32)
        np_edge_indices = np.arange(np_edge_vertices.shape[0], dtype=np.uint32)
        edge_geometry = BufferGeometry(attributes={
            'position': BufferAttribute(np_edge_vertices),
            'index'   : BufferAttribute(np_edge_indices)
        })
        edge_material = LineBasicMaterial(color=color, linewidth=2, fog=True)
        edge_lines = LineSegments(geometry=edge_geometry, material=edge_material)

        # Add geometries to pickable or non pickable objects
        self._displayed_pickable_objects.add(edge_lines)


    def AddShapeToScene(self,
                        shp,  # the TopoDS_Shape to be displayed
                        shape_color=default_shape_color,  # the default
                        render_edges=False,
                        edge_color=default_edge_color,
                        compute_uv_coords=False,
                        quality=1.0,
                        transparency=False,
                        opacity=1.):
        # first, compute the tesselation
        tess = Tesselator(shp)
        tess.Compute(uv_coords=compute_uv_coords,
                     compute_edges=render_edges,
                     mesh_quality=quality,
                     parallel=self._parallel)
        # get vertices and normals
        vertices_position = tess.GetVerticesPositionAsTuple()

        number_of_triangles = tess.ObjGetTriangleCount()
        number_of_vertices = len(vertices_position)

        # number of vertices should be a multiple of 3
        if number_of_vertices % 3 != 0:
            raise AssertionError("Wrong number of vertices")
        if number_of_triangles * 9 != number_of_vertices:
            raise AssertionError("Wrong number of triangles")

        # then we build the vertex and faces collections as numpy ndarrays
        np_vertices = np.array(vertices_position, dtype='float32').reshape(int(number_of_vertices / 3), 3)
        # Note: np_faces is just [0, 1, 2, 3, 4, 5, ...], thus arange is used
        np_faces = np.arange(np_vertices.shape[0], dtype='uint32')

        # set geometry properties
        buffer_geometry_properties = {'position': BufferAttribute(np_vertices),
                                      'index'   : BufferAttribute(np_faces)}
        if self._compute_normals_mode == NORMAL.SERVER_SIDE:
            # get the normal list, converts to a numpy ndarray. This should not raise
            # any issue, since normals have been computed by the server, and are available
            # as a list of floats
            np_normals = np.array(tess.GetNormalsAsTuple(), dtype='float32').reshape(-1, 3)
            # quick check
            if np_normals.shape != np_vertices.shape:
                raise AssertionError("Wrong number of normals/shapes")
            buffer_geometry_properties['normal'] = BufferAttribute(np_normals)

        # build a BufferGeometry instance
        shape_geometry = BufferGeometry(attributes=buffer_geometry_properties)

        # if the client has to render normals, add the related js instructions
        if self._compute_normals_mode == NORMAL.CLIENT_SIDE:
            shape_geometry.exec_three_obj_method('computeVertexNormals')

        # then a default material
        shp_material = MeshPhongMaterial(color=shape_color,
                                         polygonOffset=True,
                                         polygonOffsetFactor=1,
                                         polygonOffsetUnits=1,
                                         shininess=0.9,
                                         transparent=transparency,
                                         opacity=opacity,
                                         side='DoubleSide')

        # create a mesh unique id
        mesh_id = uuid.uuid4().hex

        # finally create the mash
        shape_mesh = Mesh(geometry=shape_geometry,
                          material=shp_material,
                          name=mesh_id)


        # and to the dict of shapes, to have a mapping between meshes and shapes
        self._shapes[mesh_id] = shp

        # edge rendering, if set to True
        edge_lines = None

        if render_edges:
            edges = list(map(lambda i_edge: [tess.GetEdgeVertex(i_edge, i_vert) for i_vert in range(tess.ObjEdgeGetVertexCount(i_edge))], range(tess.ObjGetEdgeCount())))
            edges = list(filter(lambda edge: len(edge) == 2, edges))

            np_edge_vertices = np.array(edges, dtype=np.float32).reshape(-1, 3)
            np_edge_indices = np.arange(np_edge_vertices.shape[0], dtype=np.uint32)
            edge_geometry = BufferGeometry(attributes={
                'position': BufferAttribute(np_edge_vertices),
                'index'   : BufferAttribute(np_edge_indices)
            })
            edge_material = LineBasicMaterial(color=edge_color, linewidth=1)
            edge_lines = LineSegments(geometry=edge_geometry, material=edge_material)

        # Add geometries to pickable or non pickable objects
        self._displayed_pickable_objects.add(shape_mesh)
        if render_edges:
            self._displayed_non_pickable_objects.add(edge_lines)


    def EraseAll(self):
        self._shapes = {}
        self._displayed_pickable_objects = Group()
        self._current_shape_selection = None
        self._current_mesh_selection = None
        self._current_selection_material = None
        self._renderer.scene = Scene(children=[])


    def Display(self):
        self._update_camera()
        scene_shp = Scene(children=[self._displayed_pickable_objects,
                                    self._displayed_non_pickable_objects,
                                    self._camera,
                                    AmbientLight(color='#101010')])

        self._renderer = Renderer(camera=self._camera,
                                  background=self._background,
                                  background_opacity=self._background_opacity,
                                  scene=scene_shp,
                                  controls=[OrbitControls(controlling=self._camera, target=self._camera_target), self._picker],
                                  width=self._size[0],
                                  height=self._size[1],
                                  antialias=True)
        # then display both 3d widgets and webui
        display(VBox([HBox(buttons), HBox([self._renderer, self.html])]))


    def __repr__(self):
        self.Display()
        #Now, I wrote some javascript(jQuery) code like this...
        js = ''' $(".im_control").on("click", function(e){
                     var icon = $(e.target)[0].className.split(" ").slice(-1)[0]
                     var kernel = IPython.notebook.kernel;
                     kernel.execute('on_image_click("' + icon + '")');
                  });'''

        #then, run the javascript...
        display(Javascript(js))
        return ""

In [ ]:
my_renderer = JupyterRenderer(parallel=True)

grid = my_renderer.DisplayGrid(7.5, 7.5, 15, 15)
my_renderer.DisplayShape(box1.toOCC(), transparency=True, opacity=0.5, render_edges=True, shape_color="#ff0000", edge_color='#000000', topo_level="Solid")
my_renderer.DisplayShape(box2.toOCC(), transparency=True, opacity=0.5, render_edges=True, shape_color="#00ff00", edge_color='#000000', topo_level="Solid")
my_renderer.DisplayShape(box3.toOCC(), transparency=True, opacity=0.5, render_edges=True, shape_color="#0000ff", edge_color='#000000', topo_level="Solid")

my_renderer

In [ ]:
js = ''' $(".im_control").on("click", function(e){
             var icon = $(e.target)[0].className.split(" ").slice(-1)[0]
             var kernel = IPython.notebook.kernel;
             kernel.execute('on_image_click("' + icon + '")');
          });'''

display(Javascript(js))

In [ ]:
my_renderer._camera_position

In [ ]:
edges

In [ ]:
camera.position

In [ ]:
scene.children[2].visible = False

In [ ]:
grid.colorGrid

In [ ]:
grid.colorCenterLine

In [ ]:
Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

In [ ]:
tess = Tesselator(box1.toOCC())

In [ ]:
tess.Compute(uv_coords=False,
                 compute_edges=True,
                 mesh_quality=1.0,
                 parallel=True)
    
vertices_position = tess.GetVerticesPositionAsTuple()
    


In [ ]:
tess.ObjGetTriangleCount(), tess.ObjGetEdgeCount()

In [ ]:
edges = list(map(lambda i_edge: [tess.GetEdgeVertex(i_edge, i_vert) 
                                         for i_vert in range(tess.ObjEdgeGetVertexCount(i_edge))], 
                         range(tess.ObjGetEdgeCount())))

In [ ]:
edges = list(filter(lambda edge: len(edge) == 2, edges))

In [ ]:
np_edge_vertices = np.array(edges, dtype=np.float32).reshape(-1, 3)
np_edge_indices = np.arange(np_edge_vertices.shape[0], dtype=np.uint32)
edge_geometry = BufferGeometry(attributes={

In [ ]:
edges

In [ ]:
[y for x in list(map(explode, edges)) for y in x]


In [ ]:
edges

In [ ]:
a = [1,2,3,4,5,6]

In [ ]:
explode(a)

In [ ]:
explode([1,2])

In [ ]:
flatten(list(map(explode, edges)))